In [9]:
import polars as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib  # 日本語描画
import holidays
from matplotlib.ticker import MaxNLocator
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import lightgbm as lgb
import os
from pathlib import Path
import statsmodels.api as sm
import re
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [10]:
# ========== 1) 入力ファイル ==========
Daily_Shoppers = r"C:\Users\sk062\Downloads\提供用_店舗日別客数.txt"
Bookstore_Code = r"C:\Users\sk062\Downloads\提供用_書店コード.txt"
Daily_sales_details = r"C:\Users\sk062\Downloads\提供用_日別販売明細.txt"

# 出力先
output_dir = r"C:\GitHub\keita_work"
Path(output_dir).mkdir(parents=True, exist_ok=True)

In [11]:
# ========== 2) 読み込み ==========
Daily_Shoppers_df = pl.read_csv(Daily_Shoppers, separator='\t')
Bookstore_Code_df = pl.read_csv(Bookstore_Code, separator='\t')
Daily_sales_details_df = pl.read_csv(Daily_sales_details, separator='\t')

In [12]:
comic_research = pl.read_csv(r"C:\Users\sk062\Downloads\comic_research_saved.csv" ,encoding="shift-jis")
nobel_research = pl.read_csv(r"C:\Users\sk062\Downloads\nobel_research_saved.csv" ,encoding="shift-jis")

In [13]:
Daily_Shoppers_df_copy = Daily_Shoppers_df.clone()
Bookstore_Code_df_copy = Bookstore_Code_df.clone() 
Daily_sales_details_df_copy = Daily_sales_details_df.clone() 

In [14]:
# カテゴリ列を付与
Daily_sales_details_df_copy = Daily_sales_details_df_copy.with_columns([
    pl.when(pl.col("大分類").str.contains("文庫"))
      .then(pl.lit("小説"))
      .when(pl.col("大分類").str.contains("コミック"))
      .then(pl.lit("漫画"))
      .otherwise(pl.lit("その他"))
      .alias("カテゴリ")
])

In [15]:
Daily_sales_details_df_copy = Daily_sales_details_df_copy.with_columns([
    pl.col("大分類").str.contains("文庫").fill_null(False).cast(pl.Int8).alias("is_novel"),
    pl.col("大分類").str.contains("コミック").fill_null(False).cast(pl.Int8).alias("is_comic"),
])

novel_sales = (
    Daily_sales_details_df_copy
    .filter(pl.col("is_novel") == 1)
    .select(pl.col("POS販売冊数").cast(pl.Int64).sum().alias("合計"))
)["合計"][0] or 0

comic_sales = (
    Daily_sales_details_df_copy
    .filter(pl.col("is_comic") == 1)
    .select(pl.col("POS販売冊数").cast(pl.Int64).sum().alias("合計"))
)["合計"][0] or 0

In [16]:
# --- 売上金額計算 ---
Daily_sales_details_df_copy = Daily_sales_details_df_copy.with_columns([
    (pl.col("本体価格") * pl.col("POS販売冊数")).alias("売上金額")
])

In [18]:
# ▼ 小説・漫画の年間売上金額を計算
novel_sales_amt = (
    Daily_sales_details_df_copy
    .filter(pl.col("is_novel") == 1)
    ["売上金額"]
    .sum()
)

comic_sales_amt = (
    Daily_sales_details_df_copy
    .filter(pl.col("is_comic") == 1)
    ["売上金額"]
    .sum()
)

In [19]:
# ▼ 文字列の日付を Datetime 型に変換
Daily_sales_details_df_copy = Daily_sales_details_df_copy.with_columns([
    pl.col("日付").str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S%.f", strict=False).alias("日付_dt")
])

In [20]:
# ▼ 年・月・日を抽出して列追加
Daily_sales_details_df_copy = Daily_sales_details_df_copy.with_columns([
    pl.col("日付_dt").dt.year().alias("年"),
    pl.col("日付_dt").dt.month().alias("月"),
    pl.col("日付_dt").dt.day().alias("日")
])

In [32]:
# ▼ 小説・漫画ごとの日別売上を集計
Day_sales = (
    Daily_sales_details_df_copy
    .group_by(["年", "月", "日"])
    .agg([
         (pl.col("売上金額") * pl.col("is_novel")).sum().alias("小説_売上金額"),
         (pl.col("売上金額") * pl.col("is_comic")).sum().alias("漫画_売上金額"),
         (pl.col("POS販売冊数") * pl.col("is_novel")).sum().alias("小説_販売冊数"),
         (pl.col("POS販売冊数") * pl.col("is_novel")).sum().alias("漫画_販売冊数")
    ])
    .sort(["年", "月", "日"])
)

In [33]:
comic_research = comic_research.with_columns(
    pl.col('日').str.to_datetime("%Y/%m/%d")  # 日付の形式に合わせてフォーマットを指定
)
nobel_research = nobel_research.with_columns(
    pl.col('年月日').str.to_datetime("%Y/%m/%d")  # 日付の形式に合わせてフォーマットを指定
)

SchemaError: invalid series dtype: expected `String`, got `i8` for series with name `日`

In [ ]:
# フラグ側に結合キー(年, 月, 日)を作成
comic_research = comic_research.with_columns([
    pl.col("日").dt.year().alias("年"),
    pl.col("日").dt.month().alias("月"),
    pl.col("日").dt.day().alias("日"),
])

# フラグ側に結合キー(年, 月, 日)を作成
nobel_research = nobel_research.with_columns([
    pl.col('年月日').dt.year().alias("年"),
    pl.col('年月日').dt.month().alias("月"),
    pl.col('年月日').dt.day().alias("日"),
])

In [ ]:
select_columns_nobel = [
 'レディース文庫: (日本)',
 '日本文庫: (日本)',
 '官能文庫: (日本)',
 '特殊文庫: (日本)',
 '学術・教養: (日本)',
 '海外文学: (日本)',
 '雑学文庫: (日本)',
 '年',
 '月',
 '日']

Day_sales = Day_sales.join(nobel_research.select(select_columns_nobel), on=["年", "月", "日"], how="left")

In [ ]:
select_columns_comic = [
'日',
 'マニア',
 'レディース',
 '児童',
 '少女（中高生・一般）',
 '少女（小中学生）',
 '少年（中高生・一般）',
 '少年（小中学生）',
 '廉価版',
 '成人',
 '耽美',
 '青年（一般）',
 '青年（中高年）',
 'ｗｅｂ発',
 '年',
 '月'    
]

Day_sales = Day_sales.join(comic_research.select(select_columns_comic), on=["年", "月", "日"], how="left")

In [ ]:
# ========== 新しい関数の定義 ==========
def calculate_top_x_share_flag(df, category_col, top_x, percent, new_col_name):
    """
    指定されたカテゴリ（小説 or 漫画）において、
    日ごとの販売数Top X冊の合計が、そのカテゴリの日の全体販売数の percent% を超えているかを判定する。
    1: 超えている, 0: 超えていない
    """
    
    # 1. 対象カテゴリに絞り込み、日別・書名別に集計
    target_df = df.filter(pl.col(category_col) == 1)
    daily_book_sales = target_df.group_by(["日付_dt", "書名"]).agg(
        pl.col("POS販売冊数").sum().alias("販売数")
    )
    
    # 2. 日別の「全体販売数」を算出
    daily_total = daily_book_sales.group_by("日付_dt").agg(
        pl.col("販売数").sum().alias("全体販売数")
    )
    
    # 3. 日別の「Top X冊の合計販売数」を算出
    # (日付ごとに販売数降順でソートし、上位X件を取得して合計)
    daily_top_x = (
        daily_book_sales
        .sort(["日付_dt", "販売数"], descending=[False, True])
        .group_by("日付_dt")
        .agg(pl.col("販売数").head(top_x).sum().alias("TopX合計販売数"))
    )
    
    # 4. 全体とTop Xを結合し、割合を計算してフラグ(0/1)化
    result_df = (
        daily_total
        .join(daily_top_x, on="日付_dt", how="left")
        .with_columns([
            # 割合判定: TopX合計 / 全体 > percent * 0.01
            ((pl.col("TopX合計販売数") / pl.col("全体販売数")) > (percent * 0.01))
            .cast(pl.Int8) # Booleanを0,1に変換
            .fill_null(0)  # 売上がない日は0
            .alias(new_col_name)
        ])
        .select(["日付_dt", new_col_name]) # 必要な列だけ残す
    )
    
    return result_df

# ========== 関数の実行 ==========
# 例: 小説で、Top 10冊が売上の 10% より高いか (フラグ列名: novel_top5_over30)
novel_flag_df = calculate_top_x_share_flag(
    Daily_sales_details_df_copy, 
    category_col="is_novel", 
    top_x=10, 
    percent=10, 
    new_col_name="novel_top10_over10"
)

# 例: 漫画で、Top 10冊が売上の 10% より高いか (フラグ列名: comic_top10_over50)
comic_flag_df = calculate_top_x_share_flag(
    Daily_sales_details_df_copy, 
    category_col="is_comic", 
    top_x=10, 
    percent=10, 
    new_col_name="comic_top10_over10"
)

In [ ]:
# フラグ側に結合キー(年, 月, 日)を作成
novel_flag_df = novel_flag_df.with_columns([
    pl.col("日付_dt").dt.year().alias("年"),
    pl.col("日付_dt").dt.month().alias("月"),
    pl.col("日付_dt").dt.day().alias("日"),
])

In [ ]:
comic_flag_df = comic_flag_df.with_columns([
    pl.col("日付_dt").dt.year().alias("年"),
    pl.col("日付_dt").dt.month().alias("月"),
    pl.col("日付_dt").dt.day().alias("日"),
])

In [ ]:
# Day_sales に結合 (Polarsでの結合)
Day_sales = Day_sales.join(novel_flag_df.select(["年", "月", "日", "novel_top10_over10"]), on=["年", "月", "日"], how="left")
Day_sales = Day_sales.join(comic_flag_df.select(["年", "月", "日", "comic_top10_over10"]), on=["年", "月", "日"], how="left")

In [ ]:
Day_sales.columns

['年',
 '月',
 '日',
 '小説_売上金額',
 '漫画_売上金額',
 '小説_販売冊数',
 '画_販売冊数',
 'レディース文庫: (日本)',
 '日本文庫: (日本)',
 '官能文庫: (日本)',
 '特殊文庫: (日本)',
 '学術・教養: (日本)',
 '海外文学: (日本)',
 '雑学文庫: (日本)',
 'マニア',
 'レディース',
 '児童',
 '少女（中高生・一般）',
 '少女（小中学生）',
 '少年（中高生・一般）',
 '少年（小中学生）',
 '廉価版',
 '成人',
 '耽美',
 '青年（一般）',
 '青年（中高年）',
 'ｗｅｂ発',
 'novel_top10_over10',
 'comic_top10_over10']

In [ ]:
Day_sales = Day_sales["年","月","日",'小説_販売冊数', "画_販売冊数"].head()

年,月,日,小説_売上金額,漫画_売上金額,小説_販売冊数,画_販売冊数,レディース文庫: (日本),日本文庫: (日本),官能文庫: (日本),特殊文庫: (日本),学術・教養: (日本),海外文学: (日本),雑学文庫: (日本),マニア,レディース,児童,少女（中高生・一般）,少女（小中学生）,少年（中高生・一般）,少年（小中学生）,廉価版,成人,耽美,青年（一般）,青年（中高年）,ｗｅｂ発,novel_top10_over10,comic_top10_over10
i32,i8,i8,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i8,i8
2024,1,1,602909.0,1.196768e6,807,807,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2024,1,2,849149.0,2.043955e6,1144,1144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,37,0,0,0,0,0,0
2024,1,3,1.251771e6,2.3524e6,1671,1671,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27,0,0,0,0,0,0
2024,1,4,1.531208e6,5.106318e6,2015,2015,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,45,0,0,0,0,1,1
2024,1,5,1.398347e6,3.624194e6,1862,1862,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27,0,0,0,0,1,1
